In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import pickle

# Load trained model
model = tf.keras.models.load_model("crop_disease_model.h5")
feature_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)  # last layer se pehle

# Dataset folder
dataset_path = "dataset"
classes = os.listdir(dataset_path)
train_features = {}

for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    img_files = [f for f in os.listdir(cls_path) if f.endswith(('.jpg','.png','.jpeg'))]
    
    if len(img_files) == 0:
        continue
    
    # Take first image as representative
    img_path = os.path.join(cls_path, img_files[0])
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)/255.0

    # Get feature vector
    feat = feature_model.predict(img_array).flatten()
    train_features[cls] = feat

# Save features
with open("train_features.pkl", "wb") as f:
    pickle.dump(train_features, f)

print("train_features.pkl created successfully!")


1/1 [==============================] - 0s 93ms/step
train_features.pkl created successfully!


In [2]:
import os
import pickle
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# Folder containing dataset
DATASET_DIR = "dataset/"

# Initialize pre-trained model for feature extraction (MobileNetV2, no top)
model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

features = []
labels = []

# Loop through each class folder
for class_name in os.listdir(DATASET_DIR):
    class_path = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    # Loop through images in class folder
    for img_file in os.listdir(class_path):
        if img_file.lower().endswith((".jpg", ".jpeg", ".png", ".webp")):
            img_path = os.path.join(class_path, img_file)
            
            # Load and preprocess image
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            # Extract features
            feature_vector = model.predict(img_array)
            features.append(feature_vector.flatten())
            labels.append(class_name)

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Save features and labels
with open("train_features.pkl", "wb") as f:
    pickle.dump(features, f)

with open("train_labels.pkl", "wb") as f:
    pickle.dump(labels, f)

print("train_features.pkl and train_labels.pkl generated successfully!")


1/1 [==============================] - 0s 285ms/step
train_features.pkl and train_labels.pkl generated successfully!
